### Retrieval of Relative Humidity

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

# importing data from S3
project_dir = ("https://s3groupaustralia.s3.eu-central-1.amazonaws.com/data/relative+humidity/")

In [6]:
#read world's latitude and longitude data
df = pd.read_csv(project_dir+"latAndlon.csv200303.csv")
df = df.drop('Unnamed: 0', axis = 1)

In [7]:
column_name = []
for i in range(1440):
    if i in range(0,720):
        column_name.append(str(i*0.25))
    elif i in range(720,1440):
        column_name.append('-'+str(i*0.25-180))
row_name = []
for i in range(721):
    row_name.append(str(90-i*0.25))

In [8]:
#rename the columns and rows' names by their geographical data
df.columns = [column_name]
df.index = [row_name]

In [9]:
#read heat wave data
HW_df = pd.read_excel(project_dir+"HeatWaves.xlsx")

In [10]:
#select and rename the data set we need
pos1_df = HW_df[['DisNo','Country','City', 'Lat', 'Lon']]
pos1_df.columns = ['Dis No', 'Country','City','Lat','Long']

In [11]:
lat = pos1_df['Lat']
long = pos1_df['Long']
Lat = []
Long = []

for i in range(len(lat)):
    Lat.append(lat[i])

for i in range(len(Lat)):
    string_lat = str(Lat[i])
    part = string_lat.partition('.')[0]+'.0'
    Lat[i] = pd.to_numeric(part)
    
for i in range(len(long)):
    Long.append(long[i])

for i in range(len(long)):
    string_long = str(Long[i])
    part = string_long.partition('.')[0]+'.0'
    Long[i] = pd.to_numeric(part)
     

In [12]:
column_name = []
for i in range(1440):
    if i in range(0,720):
        column_name.append(str(i*0.25))
    elif i in range(720,1440):
        column_name.append('-'+str(i*0.25-180))
row_name = []
for i in range(721):
    row_name.append(str(90-i*0.25))

In [13]:
#create a dataframe consists only column names of our final output
column_names = ['Dis No', 'Country','Year','Month','Rel_Hum']
hum_by_month = pd.DataFrame(columns = column_names)
hum_by_month

,Dis No,Country,Year,Month,Rel_Hum


We calculate each year separately, take 2016 as an example:

In [14]:
#create a dataframe having the same size as our final output but filled with value 0
import numpy as np
year = ['2016'] #select a year from 2000 to 2016
#year = ['2000', 2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016',
#             '2017','2018','2019','2020']
column_names = ['Dis No', 'Country','Year','Month','Rel_Hum']
hum_by_month = pd.DataFrame(0, index=np.arange(len(pos1_df['Lat'])*12*len(year)), columns=column_names)

In [ ]:
#extract a city's humidity from 2000.01 to 2020.12 and input the data in the dataframe we created
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in range(len(pos1_df['Lat'])):
    for j in range(len(year)):
        for k in range(len(month)):
            hum_by_month['Dis No'][i*12*len(year)+j*12+k] = pos1_df['Dis No'][i]
            hum_by_month['Country'][i*12*len(year)+j*12+k] = pos1_df['Country'][i]
            hum_by_month['Year'][i*12*len(year)+j*12+k] = year[j]
            hum_by_month['Month'][i*12*len(year)+j*12+k] = month[k]
            hum_df0 = pd.read_csv(project_dir+"RelativeHumidity.csv2016"+month[k]+".csv")
            hum_df0 = hum_df0.drop('Unnamed: 0', axis = 1)
            hum_df0.columns = [column_name]
            hum_df0.index = [row_name]
            hum0 = hum_df0.loc[str(Lat[i]),str(Long[i])]
            hum0 = str(hum0.iloc[0].iat[0])
            hum0 = pd.to_numeric(hum0[:5])
            hum_by_month['Rel_Hum'][i*12*len(year)+j*12+k] = hum0
                

In [17]:
#save the dataframe
import os

# set output path
os.chdir("/Users/zhuziyue/Desktop/MDA_project/data/")
hum_by_month.to_csv('hum_2016.csv', index=False)

In [18]:
hum_by_month

,Dis No,Country,Year,Month,Rel_Hum
0,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,1,72
1,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,2,74
2,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,3,74
3,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,4,82
4,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,5,86
...,...,...,...,...,...
13003,0,0,0,0,0
13004,0,0,0,0,0
13005,0,0,0,0,0
13006,0,0,0,0,0


### Collapse the data

Collapse the data by year, month, and disaster number.

In [20]:
#calculate the mean value within the same Disaster Number, merge with mean value dataframe, and drop duplicated rows
#and obtained the relative humidity values of every city in the heat wave data set in each month between 2000 and 2001
hd_2016 = pd.read_csv("hum_2016.csv")
grouped_dis_no = hd_2016.groupby(["Dis No","Year","Month"])
mean_hum = grouped_dis_no.mean()
mean_hum = mean_hum.reset_index()
hmd_2 = hd_2016[["Dis No", "Country"]].merge(mean_hum[["Dis No", "Year","Month",'Rel_Hum']], 
                                     on = "Dis No", 
                                     how = "left")
hmd_2 = hmd_2.drop_duplicates()
hmd_2.to_csv('hd_2016.csv', index=True, header=True)

In [24]:
hmd_2

,Dis No,Country,Year,Month,Rel_Hum
0,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,1,72.0
1,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,2,74.0
2,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,3,74.0
3,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,4,82.0
4,2020-0530-GBR,United Kingdom of Great Britain and Northern I...,2016,5,86.0
...,...,...,...,...,...
48536,2014-0010-AUS,Australia,2016,9,33.0
48537,2014-0010-AUS,Australia,2016,10,25.0
48538,2014-0010-AUS,Australia,2016,11,29.0
48539,2014-0010-AUS,Australia,2016,12,28.0
